In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import eurostat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [27]:
### CONSTANTS

# Columns that identify the data
ID_COLS = ['freq', 'unit', 'tra_meas', 'airp_pr\\TIME_PERIOD']

# Minimum fraction of non-NaN values required to keep a row
FRACTION_NON_NAN = 0.5

avia_par_countries = [
    "be", "bg", "dk", "de", "ee", "ie", "el", "es", "fr", "hr",
    "it", "cy", "lv", "lt", "lu", "hu", "mt", "nl", "at", "pl",
    "pt", "ro", "si", "sk", "fi", "se", "is", "no", "ch", "uk",
    "ba", "me", "mk", "rs", "tr"
]

In [8]:
data = eurostat.get_data_df("avia_par_de")
data.head(10)

,freq,unit,tra_meas,airp_pr\TIME_PERIOD,1993,1994,1995,1996,1997,1998,...,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2024-Q1,2024-Q2,2024-Q3,2024-Q4
0,A,FLIGHT,CAF_PAS,DE_ED00_DE_EDDB,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,FLIGHT,CAF_PAS,DE_ED00_DE_EDDH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,FLIGHT,CAF_PAS,DE_ED00_DE_EDDT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,FLIGHT,CAF_PAS,DE_EDDB_AE_OMAA,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,FLIGHT,CAF_PAS,DE_EDDB_AL_LATI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A,FLIGHT,CAF_PAS,DE_EDDB_AT_LOWS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,A,FLIGHT,CAF_PAS,DE_EDDB_AT_LOWW,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A,FLIGHT,CAF_PAS,DE_EDDB_BE_EBBR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,A,FLIGHT,CAF_PAS,DE_EDDB_BG_LBBG,284.0,601.0,228.0,317.0,295.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,A,FLIGHT,CAF_PAS,DE_EDDB_BG_LBSF,7.0,7.0,14.0,6.0,21.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# # filter rows where unit is 'PAS' and freq is 'M'
# data_pas_m =data[(data['unit'] == 'PAS') & (data['freq'] == 'M')]


In [34]:
def get_monthly_data(data, year): 
    """ Get monthly data for passenger traffic for a given year """
    year_df = data[
        ID_COLS +
        data.columns[data.columns.str.match(fr'^{year}($|-)', na=False)].tolist()
    ]
    if year_df.shape[1] != 21: ## 21 columns: 4 ID_COLS + 12 months + 4 quarters + id column
        raise ValueError(f"Expected 21 columns for year {year}, got {year_df.shape[1]}")
    year_m_pas_pasbrd =year_df[(year_df['unit'] == 'PAS') & (year_df['freq'] == 'M') & (year_df['tra_meas'] == 'PAS_BRD')]

    # 1. Select columns matching a regex (example: date columns)
    regex = r'^\d{4}'   # adjust as needed
    cols = year_m_pas_pasbrd.filter(regex=regex).columns

    # 2. Keep rows where more than half of those columns are non-NaN
    threshold = len(cols) * FRACTION_NON_NAN
    df_filtered = year_m_pas_pasbrd[year_m_pas_pasbrd[cols].notna().sum(axis=1) > threshold]
    print(f"Removed {year_m_pas_pasbrd.shape[0] - df_filtered.shape[0]} rows with insufficient data for year {year}")
    print(f"Remaining rows: {df_filtered.shape[0]}")

    # sanity check. do all rows have an airport route according to parsable regex?
    if not df_filtered['airp_pr\\TIME_PERIOD'].str.match(r'^[A-Z]{2}_[A-Z]{2}.._[A-Z]{2}_[A-Z]{2}..$', na=False).all():
        raise ValueError(f"Not all rows have a valid airport route for year {year}")
    
    # are all rows unique according to airport route?
    if df_filtered['airp_pr\\TIME_PERIOD'].nunique() != df_filtered.shape[0]:
        raise ValueError(f"Not all rows are unique according to airport route for year {year}")

    return df_filtered


In [35]:
d1 = get_monthly_data(data, 2015)
d1.head(10)
print(d1.shape)

Removed 1171 rows with insufficient data for year 2015
Remaining rows: 861
(861, 21)
